In [1]:
import sparknlp
spark=sparknlp.start()

In [2]:
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import pandas as pd

In [3]:
query="SELECT * FROM train_1_csv"
traindata=spark.sql(query)

In [4]:
traindata.show(5)

+---+-------+--------+--------------------+-------+
 id|keyword|location| text|target|
+---+-------+--------+--------------------+-------+
 1| null| null|Our Deeds are the...| 1|
 4| null| null|Forest fire near ...| 1|
 5| null| null|All residents ask...| 1|
 6| null| null|13,000 people rec...| 1|
 7| null| null|Just got sent thi...| 1|
+---+-------+--------+--------------------+-------+
only showing top 5 rows

In [5]:
traindata=traindata.na.drop()

In [6]:
traindata.select("text").show(5)

+--------------------+
 text|
+--------------------+
@bbcmtd Wholesale...|
We always try to ...|
#AFRICANBAZE: Bre...|
Crying out for mo...|
On plus side LOOK...|
+--------------------+
only showing top 5 rows

In [7]:
traindata.select("target\r").show(5)

+-------+
target|
+-------+
 1|
 0|
 1|
 0|
 0|
+-------+
only showing top 5 rows

In [8]:
#only train and target column are useful for analysis

In [9]:
#changing name of target column
traindata=traindata.withColumnRenamed('target\r', 'target')

In [10]:
#removing any type of whitespace from target column
from pyspark.sql.functions import *
traindata = traindata.withColumn('target', regexp_replace('target', '\r', ''))

In [11]:
from pyspark.sql.functions import col
traindata.groupBy("target") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+------+-----+
target|count|
+------+-----+
 0| 2884|
 1| 2196|
+------+-----+

In [12]:
traindata.columns

Out[12]: ['id', 'keyword', 'location', 'text', 'target']

In [13]:
traindata=traindata.drop('id', 'keyword', 'location')

In [14]:
traindata.show(5)

+--------------------+------+
 text|target|
+--------------------+------+
@bbcmtd Wholesale...| 1|
We always try to ...| 0|
#AFRICANBAZE: Bre...| 1|
Crying out for mo...| 0|
On plus side LOOK...| 0|
+--------------------+------+
only showing top 5 rows

In [15]:
#creating nlp pipeline using spark-nlp

In [16]:
document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")


In [17]:
sentence_detector = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

In [18]:
tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

In [19]:
normalizer = Normalizer()\
    .setInputCols(["token"])\
    .setOutputCol("normalized")

In [20]:
stopwords_cleaner=StopWordsCleaner()\
  .setInputCols("normalized")\
  .setOutputCol("cleanTokens")\
  .setCaseSensitive(False)

In [21]:
lemma=LemmatizerModel.pretrained('lemma_antbnc')\
  .setInputCols(["cleanTokens"])\
  .setOutputCol("lemma")

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[ | ][ / ][ — ][OK!]

In [22]:
word_embeddings=WordEmbeddingsModel().pretrained()\
  .setInputCols(["document","lemma"])\
  .setOutputCol("embeddings")\
  .setCaseSensitive(False)

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]

In [23]:
embeddingsSentence=SentenceEmbeddings()\
  .setInputCols(["document","embeddings"])\
  .setOutputCol("sentence_embeddings")\
  .setPoolingStrategy("AVERAGE")

In [24]:
classifierdl=ClassifierDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")\
  .setLabelColumn("target")\
  .setMaxEpochs(5)\
  .setEnableOutputLogs(True)

In [25]:
clf_pipeline=Pipeline(
    stages=[document,
            tokenizer,
            normalizer,
            stopwords_cleaner,
            lemma,
            word_embeddings,
            embeddingsSentence,
            classifierdl])

In [26]:
clf_pipelineModel=clf_pipeline.fit(traindata)

In [27]:
df=clf_pipelineModel.transform(traindata)

In [28]:
result=df.select("class.result")

In [29]:
result.show()

+------+
result|
+------+
 [0]|
 [0]|
 [1]|
 [0]|
 [0]|
 [0]|
 [1]|
 [0]|
 [0]|
 [0]|
 [1]|
 [0]|
 [0]|
 [1]|
 [0]|
 [0]|
 [1]|
 [0]|
 [1]|
 [1]|
+------+
only showing top 20 rows

In [30]:
clf_pipelineModel.stages

Out[99]: [DocumentAssembler_9b240950b42a,
 REGEX_TOKENIZER_3ed7a0c8799d,
 NORMALIZER_a7553e6b3e30,
 StopWordsCleaner_8ac7552e956e,
 LEMMATIZER_c62ad8f355f9,
 WORD_EMBEDDINGS_MODEL_48cffc8b9a76,
 SentenceEmbeddings_9e1d9eb85ca0,
 ClassifierDLModel_050739262419]

In [31]:
from sklearn.metrics import classification_report,accuracy_score
df=clf_pipelineModel.transform(traindata).select('text','target',"class.result").toPandas()
df['result']=df['result'].apply(lambda x:x[0])
print(classification_report(df.target,df.result))
print(accuracy_score(df.target,df.result))

precision recall f1-score support

 0 0.83 0.90 0.86 2884
 1 0.85 0.76 0.80 2196

 micro avg 0.84 0.84 0.84 5080
 macro avg 0.84 0.83 0.83 5080
weighted avg 0.84 0.84 0.84 5080

0.8387795275590552

In [32]:
###lets use UniversalSentenseEncoder

In [33]:
use = UniversalSentenceEncoder.pretrained()\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]

In [34]:
classsifierdl = ClassifierDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")\
  .setLabelColumn("target")\
  .setMaxEpochs(5)\
  .setEnableOutputLogs(True)

In [35]:
use_clf_pipeline = Pipeline(
    stages = [
        document,
        use,
        classsifierdl
    ])

In [36]:
use_pipelineModel = use_clf_pipeline.fit(traindata)

In [37]:
from sklearn.metrics import classification_report,accuracy_score
df=clf_pipelineModel.transform(traindata).select('text','target',"class.result").toPandas()
df['result']=df['result'].apply(lambda x:x[0])
print(classification_report(df.target,df.result))
print(accuracy_score(df.target,df.result))

precision recall f1-score support

 0 0.83 0.90 0.86 2884
 1 0.85 0.76 0.80 2196

 micro avg 0.84 0.84 0.84 5080
 macro avg 0.84 0.83 0.83 5080
weighted avg 0.84 0.84 0.84 5080

0.8387795275590552